# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [28]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import warnings
import numpy as np
from pprint import pprint
warnings.filterwarnings("ignore")

# Import API key
from api_keys import geoapify_key

In [3]:
import cartopy.crs as ccrs
import geoviews as gv # noqa

In [4]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,nova sintra,14.8667,-24.7167,20.73,77,0,4.07,CV,1683558969
1,1,hammerfest,70.6634,23.6821,1.25,93,75,2.57,NO,1683558970
2,2,xichang,27.8964,102.2634,16.88,73,52,2.57,CN,1683558970
3,3,markaz-e hukumat-e darweshan,31.1323,64.1934,28.88,7,0,4.96,AF,1683558971
4,4,ayr,55.4627,-4.6339,15.17,100,100,4.12,GB,1683558971


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [5]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points('Lng','Lat',geo=True, tiles='EsriImagery', size= 'Humidity', scale= .5, frame_width = 800,
    frame_height = 600, color= 'r')

# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [13]:
# Narrow down cities that fit criteria and drop any results with null values
perfect_destination_df = city_data_df.loc[(city_data_df['Max Temp'] > 21) & (city_data_df['Max Temp'] <= 27) & 
                                         (city_data_df['Wind Speed'] < 4.5) & (city_data_df['Cloudiness'] == 0)]



# Drop any rows with null values
perfect_destination_df.dropna()

# Display sample data
perfect_destination_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
28,28,calama,-22.4667,-68.9333,21.27,13,0,3.60,CL,1683558982
47,47,moranha,27.1833,94.9333,22.57,64,0,1.60,IN,1683558990
51,51,les avirons,-21.2418,55.3394,25.05,65,0,1.03,RE,1683558888
72,72,birjand,32.8663,59.2211,26.26,6,0,3.09,IR,1683558998
101,101,hamilton,39.1834,-84.5333,24.25,71,0,2.57,US,1683559005
108,108,tezpur,26.6333,92.8000,25.70,56,0,3.08,IN,1683558943
133,133,vallenar,-28.5708,-70.7581,26.70,12,0,1.58,CL,1683559014
198,198,tura,25.5198,90.2201,26.05,30,0,2.11,IN,1683559040
378,378,torbat-e heydariyeh,35.2740,59.2195,24.18,16,0,3.85,IR,1683559118
441,441,taltal,-25.4000,-70.4833,21.29,59,0,2.46,CL,1683559140


### Step 3: Create a new DataFrame called `hotel_df`.

In [21]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
    ## note to self: "Why use copy function instead of just assigning new variable?": 
    ### https://stackoverflow.com/questions/27673231/why-should-i-make-a-copy-of-a-data-frame-in-pandas

hotel_df = perfect_destination_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
28,calama,CL,-22.4667,-68.9333,13,
47,moranha,IN,27.1833,94.9333,64,
51,les avirons,RE,-21.2418,55.3394,65,
72,birjand,IR,32.8663,59.2211,6,
101,hamilton,US,39.1834,-84.5333,71,
108,tezpur,IN,26.6333,92.8000,56,
133,vallenar,CL,-28.5708,-70.7581,12,
198,tura,IN,25.5198,90.2201,30,
378,torbat-e heydariyeh,IR,35.2740,59.2195,16,
441,taltal,CL,-25.4000,-70.4833,59,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [80]:
### Source code: [2] Geoapify methods: edX Boot Camps LLC. Module 6, Activity 5: airports.ipynb
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
    "limit":1
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
# pprint(name_address)
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
calama - nearest hotel: Apart Hotel Geotel Calama
moranha - nearest hotel: No hotel found
les avirons - nearest hotel: Floralys
birjand - nearest hotel: مهمانسرای جهانگردی بیرجند
hamilton - nearest hotel: North Vista Manor
tezpur - nearest hotel: Hotel Basant
vallenar - nearest hotel: Hotel del Marqués
tura - nearest hotel: No hotel found
torbat-e heydariyeh - nearest hotel: هتل رستوران سنتی لاری
taltal - nearest hotel: Hostería Taltal
nishon tumani - nearest hotel: Жура ота
kailua-kona - nearest hotel: Kona Seaside Hotel
upington - nearest hotel: Die Eiland Holiday Ressort
shibirghan - nearest hotel: Attayee Hotel and Guesthouse


,City,Country,Lat,Lng,Humidity,Hotel Name
28,calama,CL,-22.4667,-68.9333,13,Apart Hotel Geotel Calama
47,moranha,IN,27.1833,94.9333,64,No hotel found
51,les avirons,RE,-21.2418,55.3394,65,Floralys
72,birjand,IR,32.8663,59.2211,6,مهمانسرای جهانگردی بیرجند
101,hamilton,US,39.1834,-84.5333,71,North Vista Manor
108,tezpur,IN,26.6333,92.8000,56,Hotel Basant
133,vallenar,CL,-28.5708,-70.7581,12,Hotel del Marqués
198,tura,IN,25.5198,90.2201,30,No hotel found
378,torbat-e heydariyeh,IR,35.2740,59.2195,16,هتل رستوران سنتی لاری
441,taltal,CL,-25.4000,-70.4833,59,Hostería Taltal


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [83]:
%%capture --no-display
### Formatting source [3]: https://docs.bokeh.org/en/latest/docs/user_guide/styling/visuals.html#ug-styling-colors

# Configure the map plot
vacation_plot = hotel_df.hvplot.points('Lng','Lat',geo=True, tiles='EsriImagery', size= 'Humidity', frame_width = 750,
                frame_height = 600, color= 'City', scale = 1.5, hover_cols= ['Hotel Name', 'Country'], 
                hover_fill_color='r', line_color= 'white', fill_alpha= .75, hover_fill_alpha= 1)

# Display the map
vacation_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)